In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import math
import json
import ast
import requests
import os
import statsmodels.formula.api as smf


In [50]:
characters = pd.read_csv('../Data/MovieSummaries/character.metadata.tsv', sep='\t', header=None, 
    names = ["WikiID", "FreebaseID", "Release", "Character name", "Birth", 
    "Sex", "Height (m)", "Ethnicity ID", "Name", "Age at movie release",
    "Freebase character/actor map ID", "Freebase character ID", "Freebase actor ID"])
characters.head()

,WikiID,FreebaseID,Release,Character name,Birth,Sex,Height (m),Ethnicity ID,Name,Age at movie release,Freebase character/actor map ID,Freebase character ID,Freebase actor ID
0,975900,/m/03vyhn,2001-08-24,Akooshay,1958-08-26,F,1.620,NaN,Wanda De Jesus,42.0,/m/0bgchxw,/m/0bgcj3x,/m/03wcfv7
1,975900,/m/03vyhn,2001-08-24,Lieutenant Melanie Ballard,1974-08-15,F,1.780,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4
2,975900,/m/03vyhn,2001-08-24,Desolation Williams,1969-06-15,M,1.727,/m/0x67,Ice Cube,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l
3,975900,/m/03vyhn,2001-08-24,Sgt Jericho Butler,1967-09-12,M,1.750,NaN,Jason Statham,33.0,/m/02vchl6,/m/0bgchnq,/m/034hyc
4,975900,/m/03vyhn,2001-08-24,Bashira Kincaid,1977-09-25,F,1.650,NaN,Clea DuVall,23.0,/m/02vbb3r,/m/0bgchp9,/m/01y9xg


In [72]:
#List of all ethnicities in the dataset
ethnicities = characters['Ethnicity ID'].dropna().unique().tolist()
print(ethnicities)

['/m/044038p', '/m/0x67', '/m/064b9n', '/m/041rx', '/m/033tf_', '/m/04gfy7', '/m/0222qb', '/m/01qhm_', '/m/0dryh9k', '/m/048sp5', '/m/04mvp8', '/m/0bzkm2', '/m/02p1pl6', '/m/0bjbszh', '/m/022fdt', '/m/0cqgdq', '/m/0ffkb4', '/m/075dhf0', '/m/01hwt', '/m/0xnvg', '/m/0dqqwy', '/m/048z7l', '/m/07bch9', '/m/09v5bdn', '/m/02w7gg', '/m/03bkbh', '/m/02vsw1', '/m/09kr66', '/m/09vc4s', '/m/0g0x7_', '/m/042gtr', '/m/0cm7w1', '/m/046cwm', '/m/04dbw3', '/m/02ctzb', '/m/0g8_vp', '/m/092h2qt', '/m/0g6ff', '/m/0278pqj', '/m/0301y_', '/m/019kn7', '/m/0cnvdq1', '/m/03295l', '/m/065b6q', '/m/03pqwy', '/m/01xhh5', '/m/03ts0c', '/m/06gbnc', '/m/07hwkr', '/m/0bpjh3', '/m/0fpjs3j', '/m/04nrnz', '/m/09k5jvk', '/m/07mqps', '/m/08hpk0', '/m/03ttfc', '/m/0d9q7j', '/m/075_n6', '/m/0dllcfn', '/m/04kbvpz', '/m/03ftx7', '/m/0747611', '/m/025rpb0', '/m/06mvq', '/m/047l_90', '/m/029f2r', '/m/01rv7x', '/m/05sf2x', '/m/01336l', '/m/0bh91q8', '/m/01g7zj', '/m/0cn68', '/m/02sch9', '/m/0fqp6zk', '/m/02y_9mh', '/m/0d7wh', '

In [53]:
ethnicities_data = pd.read_csv('../Data/ethnicities_without_clusters.tsv', sep='\t', 
                               header=None, names=["Ethnicity ID", "Ethnicity"])
ethnicities_data.head()

,Ethnicity ID,Ethnicity
0,/m/0x67,African Americans
1,/m/064b9n,Omaha people
2,/m/041rx,Jewish people
3,/m/033tf_,Irish Americans
4,/m/04gfy7,Indian Americans


In [55]:
merged_characters = characters.merge(ethnicities_data, how='inner', on='Ethnicity ID')
merged_characters.head()

,WikiID,FreebaseID,Release,Character name,Birth,Sex,Height (m),Ethnicity ID,Name,Age at movie release,Freebase character/actor map ID,Freebase character ID,Freebase actor ID,Ethnicity
0,975900,/m/03vyhn,2001-08-24,Desolation Williams,1969-06-15,M,1.727,/m/0x67,Ice Cube,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l,African Americans
1,975900,/m/03vyhn,2001-08-24,Commander Helena Braddock,1949-05-26,F,1.727,/m/0x67,Pam Grier,52.0,/m/02vdcfp,/m/0bgchnd,/m/0418ft,African Americans
2,3196793,/m/08yl5d,2000-02-16,NaN,1937-11-10,M,NaN,/m/0x67,Albert Hall,62.0,/m/0lr37dy,NaN,/m/01lntp,African Americans
3,156558,/m/014k4y,2001-06-27,Yvette,1970-09-11,F,1.650,/m/0x67,Taraji P. Henson,30.0,/m/0jtx5t,/m/03jnxj_,/m/0blbxk,African Americans
4,156558,/m/014k4y,2001-06-27,Jody,1978-12-30,M,1.797,/m/0x67,Tyrese Gibson,22.0,/m/0jtx5h,/m/03jnxf4,/m/01l1b90,African Americans


In [85]:
''' code by CLEMENT (inspired by stackoverflow)
https://stackoverflow.com/questions/74277269/wikidata-query-service-how-do-i-search-by-freebase
'''

def get_freebase_label(freebase_ids):
      labels = []
      url = 'https://query.wikidata.org/sparql'
      for id in freebase_ids:
            query = '''
            SELECT ?s ?sLabel ?freebaseID
            WHERE {

            VALUES ?freebaseID { 
            "'''+id+'''"
            }

            ?s wdt:P646 ?freebaseID .

            SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . } 
            }
                  '''
            r = requests.post(url, params = {'format': 'json', 'query': query})
            data = r.json()
            labels.append(data['results']['bindings'][0]['sLabel']['value'])
      return labels

a = ['/m/0x67', '/m/064b9n']
labels = get_freebase_label(a)
print(labels)


['African Americans', 'Omaha people']


In [5]:
if os.path.isfile("./Data/characters"):
    # To accelerate code execution we will import the pickle file if it exists
    characters = pd.read_pickle("./Data/characters")
else:
    # Function which allows query search of ids using Wikidata SPARQL service
    def get_ethnicities(ids):

        tmp = '""'.join(ids)

        url = 'https://query.wikidata.org/sparql'
        query = '''
        SELECT  ?ethnicities ?sLabel WHERE {
        VALUES ?ethnicities {"'''+tmp+'''"} 
        ?s wdt:P646 ?ethnicities .

            SERVICE wikibase:label {
            bd:serviceParam wikibase:language "en" .
            }
        }
        '''
        r = requests.post(url, params = {'format': 'json', 'query': query})
        data = r.json()
        return data
    
    d1 = get_ethnicities(ethnicities[1:200])
    d2 = get_ethnicities(ethnicities[201:478])
    tmp1 = []
    tmp2 = []
    for d in (d1, d2):
        for el in d["results"]["bindings"]:
            tmp1.append(el["ethnicities"]["value"])
            tmp2.append(el["sLabel"]["value"])
    table_eth = pd.DataFrame(tmp1, tmp2).reset_index().rename(columns = {"index": "Ethnicities", 0: "code"})
    # Table associating the freebase id with the ethnicities


    """Code for querying using Google's knowledge graph"""
    import json
    import urllib
    # We are only going to use the Google's KG for the remaining codes that we couldn't find in WikiData 
    ids = list(set(ethnicities) - set(table_eth["code"]))
    # Private api key for using Google's API
    if os.path.isfile("api_key.txt"):
        api_key = open("api_key.txt").read()
        service_url = 'https://kgsearch.googleapis.com/v1/entities:search'
        params = {
            'limit': 500,
            'indent': True,
            'key': api_key,
        }
        url = service_url + '?' + 'ids='+'&ids='.join(ids).replace("/", "%2F" )+'&'+urllib.parse.urlencode(params)
        response = json.loads(urllib.request.urlopen(url).read())
        for element in response['itemListElement']:
            print(element['result']['name'] + ' (' + str(element['result']["@id"]) + ')')
        tmp1 = []
        tmp2 = []
        for element in response['itemListElement']:
            tmp2.append(element['result']['name'])
            tmp1.append(element['result']["@id"].replace("kg:", ''))
        table_eth = pd.concat([table_eth, pd.DataFrame(tmp1, tmp2).reset_index().rename(columns = {"index": "Ethnicities", 0: "code"})])
 

    # Replace ethnicity code by value
    characters["Ethnicity"] = characters["Ethnicity"].apply(lambda x: table_eth["Ethnicities"][table_eth["code"]==x].values[0]
    if any(table_eth["code"]==x) else x)
    characters.to_pickle("./Data/characters")
characters.head()

KeyboardInterrupt: 